In [197]:

import time
from flask import request, Response
from flask import Flask
from config import netbox_api
from jinja2 import Environment, FileSystemLoader
from nornir_napalm.plugins.tasks import napalm_get, napalm_cli
import pprint
from nornir_utils.plugins.functions import print_result
from nornir.core.task import Task, Result
from nornir_jinja2.plugins.tasks import template_file
from nornir_netmiko.tasks import netmiko_send_command, netmiko_send_config
import re

from credentials import(netbox_url,
                        netbox_token,
                        device_username,
                        device_password)
from nornir import InitNornir
from nornir.core.filter import F

In [198]:
def create_nornir_session():
    """ 
    Инициализируем nornir, но для "hosts" используем данные из netbox
    :return: nr_session
    """
    nr_session = InitNornir(
        inventory={
            "plugin": "NetBoxInventory2",
            "options": {
                "nb_url": netbox_url,
                "nb_token": netbox_token,
                "group_file": "./inventory/groups.yml",
                "defaults_file": "./inventory/defaults.yml",
            },
        },
    )
    return nr_session

In [199]:
def conversion(tup, dict = {}):
    for x, y in tup:
        dict.setdefault(x, []).append(y)
    return dict

In [200]:
templates_path = "./templates/"

Заполняем шаблон

In [201]:
def cisco_config_interface(j2_interface,event='None'):
    """ 
    Заполнение шаблона значениями
    :param j2_interface: интерфейс 
    :param event: событие
    :return: None
    """   

    def template_fill(*args,**kwargs):
        environment = Environment(loader=FileSystemLoader(templates_path)) # загружаем шаблон для заполнения
        template = environment.get_template(template_file)
        content = ''
        content = template.render( # заполняем шаблон
                                    interface_name = j2_interface.name,
                                    descr = j2_interface.description,
                                    access_vlan = j2_interface.untagged_vlan.vid,
                                    mode = j2_interface.mode.value
                                )
        print("Filling in the template...\n{}".format(content)) 
        return content  
    
    if event != 'delete':    
        template_file = "cisco_ios_access_interface.template"
        content = template_fill(j2_interface, template_file)
    else:
        template_file = "cisco_ios_default_interface.template"
        content = template_fill(j2_interface, template_file)
            
    return content
#get_device_interface = netbox_api.dcim.interfaces.get(136)
#cisco_config_interface(get_device_interface, event='delete')

Получаем через napalm интерфейсы с устройства

In [202]:
def push_config_interface(netbox_interface,event='None'):
    """  
    Проверка соответствия портов между netbox и реальным устройством
    :param netbox_interface: ссылка на объект интерфейса pynetbox
    :return: None
    """
    attempts = 3 # количество попыток подключения
    timeout = 5 # время ожидания между попытками в секундах
    name = netbox_interface.name # избавляемся от пробелов и игнорируем регистр
    filter_query = '10.30.1.105'
    nr = create_nornir_session()
    sw = nr.filter(hostname = filter_query) # производим отбор по конкретному хосту

    for _ in range(attempts):
        get_int = sw.run(task=napalm_get, getters=['get_interfaces']) # получаем все интерфейсы с устройства в виде словаря
        if get_int.failed == False: 
            print('Connection state is connected...')
            break
        else:
            print('Connection state is failed...')
            time.sleep(timeout)

    for device in get_int.values():
        interfaces = device.result['get_interfaces'].keys() # получаем интерфейсы как ключи словаря
        if name in (intf for intf in list(interfaces)):
            print("Find {} for device {}".format(name, device.host))
    

    result = sw.run(netmiko_send_config,name="Configuration interface.../",config_commands=cisco_config_interface(netbox_interface,event).split('\n'))
    print_result(result)
    sw.close_connections()
    
#push_config_interface(netbox_api.dcim.interfaces.get(136))


Удаляем (сбрасываем) конфигурацию интерфейса и добавляем дефолтные настройки

In [203]:
def delete_config_intf(netbox_interface):
    
    print("Delete interface {} config...".format(netbox_interface))
    push_config_interface(netbox_interface,event='delete')

In [204]:
def create_config_intf(netbox_interface):
    """ 
    Новый интерфейс
    :param netbox_interface
    """
    print("Push new interface {} config...".format(netbox_interface))
    push_config_interface(netbox_interface,event='create')

Вносим изменения в конфигурацию интерфейса

In [205]:
def update_config_intf(netbox_interface):
    """ 
    Обновление интерфейса
    :param netbox_interface
    """
    print("Updating interface {} config...".format(netbox_interface))
    push_config_interface(netbox_interface, event='update')

In [206]:
def manage_device_interfaces():
    
    devices_keys = ['role','device_id','intf_id'] # список ключей для словаря devices
    devices = [] 
    templates_roles = ['access_switch', 'user_device'] # получаем из netbox (произвольные данные)
    device_roles = []
    regex = "[a|b]_terminations"
    
    get_cable_id = conversion(list(netbox_api.dcim.cables.get(request.json["data"]["id"])))
    #get_device_cable = conversion(list(netbox_api.dcim.cables.get('306')))
    
    for key in get_cable_id.keys(): # заполняем список device_value и объединяем с device_keys в словарь
        if re.match(regex, key): # отбираем нужные ключи из словаря по регулярке
            devices_values = []
            device_id = get_cable_id[key][0][0]['device']['id']
            devices_values.append(netbox_api.dcim.devices.get(device_id).device_role.slug) # роль устройства
            devices_values.append(device_id) # id устройства
            devices_values.append(get_cable_id[key][0][0]['id']) # id интерфейса устройства
            devices.append(dict(zip(devices_keys,devices_values))) # получаем список из словарей
            print("Dictionary append into list...")
    
    for device in devices: # заполняем список ролей
        device_roles.append(device['role'])
     
    if set(device_roles) == set(templates_roles): # проверяем, что получили устройства с разными ролями и в соответствии со списком    
        for device in devices:
            if device['role'] == templates_roles[0]: # нам нужен access switch
                device_intf_id = device['intf_id'] # получаем ID интерфейса access switchа из, нами созданного, словаря            
        get_device_interface = netbox_api.dcim.interfaces.get(device_intf_id) # по ID находим интерфейс в netbox    
        print("List is equal {}, switch access interface ID: {}...".format(device_roles, device_intf_id))
        if get_device_interface.mgmt_only: # проверяем, является ли интерфейс management интерфейсов
            print("\tManagement interface, no changes will be performed...")
        else: 
            #update_config_intf(netbox_interface=get_device_interface)
            if request.json["event"] == "deleted": # Конфиг интерфейса адрес будет удален

                    delete_config_intf(netbox_interface=get_device_interface)

            elif request.json["event"] == "created": # Конфиг интерфейса будет добавлен

                    create_config_intf(netbox_interface=get_device_interface)

            elif request.json["event"] == "updated": # Конфиг интерфейса будет изменен

                    update_config_intf(netbox_interface=get_device_interface)
            
    else: print("List is not equal")
    
        
    return Response(status=204)
#manage_device_interfaces()

In [207]:
# Create a Flask instance
app = Flask(__name__)
app.add_url_rule("/api/config_intf",
                methods=['POST'],
                view_func=manage_device_interfaces)
    
if __name__ == "__main__": 
    app.run(host='0.0.0.0', port=8080)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://10.96.5.16:8080
Press CTRL+C to quit


List is not equal


10.30.1.226 - - [19/Mar/2023 21:56:37] "POST /api/config_intf HTTP/1.1" 204 -
